In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [69]:
from exp.eventlog import *
import torch
from torch import nn
from torch.nn import init
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
import torch.nn.functional as F

import sys
import math

In [3]:
path = untar_data(URLs.BPIC_2012)
log = import_xes(path,extensions=False,classifiers=False,schema=False,log_attributes=False)

failed to parse date: 1970-01-01T00:00:00.000+01:00
failed to parse date: 1970-01-01T00:00:00.000+01:00
failed to parse date: 2012-04-23T00:00:00.000+02:00
failed to parse date: 2011-10-01T00:38:44.546+02:00
failed to parse date: 2012-03-14T16:04:54.681+01:00


In [4]:
df = log.events
df.head()

,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00
1,173688,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00
2,173688,2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00
3,173688,3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00
4,173688,4,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00


In [5]:
# convert traces generalized
def conv_traces(df):
    d = {}
    for i in df.columns:
        d[i] = ' '.join(df[i].map(str))
        
    return pd.Series(d)

In [6]:
# df = df.groupby('trace_id')[df.columns[df.columns!='trace_id']].apply(conv_traces)
# df.head()

In [7]:
__all__ = ['LMTextList', 'LMLabelList']

def label_for_lm(self, **kwargs):
    "A special labelling method for language models."
    self.__class__ = LMTextList
    kwargs['label_cls'] = LMLabelList
    return self.label_const(0, **kwargs)

In [8]:
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()
default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

In [9]:
from collections import OrderedDict

def uniqueify(x, sort=False):
    res = list(OrderedDict.fromkeys(x).keys())
    if sort: res.sort()
    return res

class Processor():
    def process(self, items): return items

class CategoryProcessor(Processor):
    def __init__(self,default_token=None): 
        self.vocab=None
        self.default_token=default_token

    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            self.vocab = uniqueify(items)
            if self.default_token is not None:
                for o in reversed(self.default_token):
                    if o in self.vocab: self.vocab.remove(o)
                    self.vocab.insert(0, o)
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return self.otoi[item]

    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return self.vocab[idx]

In [10]:
user_numericalize=CategoryProcessor(default_spec_tok)
user_numericalize(df['org:resource'])

activity_numericalize=CategoryProcessor(default_spec_tok)
activity_numericalize(df['concept:name']);

In [11]:
%%time

def create_traces(df):
    ll=[]
    trace_ids=[]
    for n, g in df.groupby('trace_id'):
        ea=activity_numericalize(g['concept:name'])
        eu=user_numericalize(g['org:resource'])
        ll.append([ea,eu])
        trace_ids.append(n)

    df2=pd.DataFrame(ll,columns=["Activities",'Users'])
    df2.index=trace_ids
    return df2

data=create_traces(df)

CPU times: user 2.78 s, sys: 821 µs, total: 2.78 s
Wall time: 2.78 s


In [13]:
data.head()

,Activities,Users
173688,"[8, 9, 10, 11, 11, 12, 13, 14, 15, 16, 17, 11,...","[8, 8, 8, 8, 9, 10, 10, 10, 10, 10, 9, 9, 9, 9..."
173691,"[8, 9, 10, 11, 11, 11, 11, 12, 14, 13, 15, 16,...","[8, 8, 8, 8, 9, 9, 9, 10, 10, 10, 10, 10, 9, 9..."
173694,"[8, 9, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12,...","[8, 8, 8, 8, 16, 16, 9, 9, 17, 17, 17, 17, 17,..."
173697,"[8, 9, 26]","[8, 8, 8]"
173700,"[8, 9, 26]","[8, 8, 8]"


In [14]:
def random_split(np_arr,train=0.9):
    shuffle = np_arr[np.random.permutation(len(np_arr))]
    split = int(len(np_arr)*0.9)
    train, valid = shuffle[:split], shuffle[split:]
    return train, valid

In [16]:
train, valid = random_split(data.values)

In [18]:
train[:5]

array([[list([8, 9, 28, 28, 28, 28, 28, 28, 10, 11, 28, 11, 11, 11, 11, 11, 11, 11, 26, 11]),
        list([8, 8, 8, 20, 20, 20, 20, 20, 20, 20, 20, 16, 16, 20, 20, 16, 16, 16, 16, 16])],
       [list([8, 9, 26]), list([8, 8, 8])],
       [list([8, 9, 10, 11, 11, 11, 11, 11, 11, 11, 11, 12, 14, 13, 15, 16, 17, 11, 17, 18, 19, 17]),
        list([8, 8, 8, 8, 75, 75, 41, 41, 65, 65, 17, 17, 17, 17, 17, 17, 17, 17, 28, 28, 28, 28])],
       [list([8, 9, 26]), list([8, 8, 8])],
       [list([8, 9, 10, 11, 11, 11, 11, 27, 11]),
        list([8, 8, 8, 8, 18, 18, 34, 34, 34])]], dtype=object)

In [32]:
# Blicke ich nicht durch

class PPDataSet():
    def __init__(self, activities,users, bs=64, bptt=70, shuffle=False):
        self.data,self.bs,self.bptt,self.shuffle = data,bs,bptt,shuffle
        total_len = sum([len(t) for t in data])
        self.n_batch = total_len // bs
        self.a_batched=self.batchify(activities)
        self.u_batched=self.batchify(users)

    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        a_source = self.a_batched[idx % self.bs]
        u_source = self.u_batched[idx % self.bs]

        seq_idx = (idx // self.bs) * self.bptt
        return ((a_source[seq_idx:seq_idx+self.bptt],u_source[seq_idx:seq_idx+self.bptt]),
                 (a_source[seq_idx+1:seq_idx+self.bptt+1],u_source[seq_idx+1:seq_idx+self.bptt+1]))
    
    def batchify(self,d):
        texts = d
        if self.shuffle: texts = texts[torch.randperm(len(texts))]
        stream = torch.cat([torch.cat((tensor([2]),tensor(t),tensor([3]))) for t in texts])
        return stream[:self.n_batch * self.bs].view(self.bs, self.n_batch)

In [38]:
train[0]

array([list([8, 9, 28, 28, 28, 28, 28, 28, 10, 11, 28, 11, 11, 11, 11, 11, 11, 11, 26, 11]),
       list([8, 8, 8, 20, 20, 20, 20, 20, 20, 20, 20, 16, 16, 20, 20, 16, 16, 16, 16, 16])],
      dtype=object)

In [37]:
dl = DataLoader(PPDataSet(activities, users, shuffle=False), batch_size=bs)
dl

NameError: name 'activities' is not defined

In [33]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Embedding(n_in, 7, padding_idx=1),nn.Linear(7,nh), nn.ReLU(), nn.Linear(nh,len(activity_numericalize.vocab))]
        
    def __call__(self, x):
        for l in self.layers: 
            x = l(x)
            print(x.shape)
        return x

In [34]:
loss_func = F.cross_entropy

In [27]:
# kaiming initialization:
nh = 50
m = 32 # number of columns in the dataset

w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [30]:
def relu(x): return x.clamp_min(0.)
def lin(x, w, b): x@w + b

In [39]:
w1 = torch.randn(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(train, w1, b1))

TypeError: unsupported operand type(s) for @: 'numpy.ndarray' and 'Tensor'

In [96]:
idx2char = ['h', 'i', 'e', 'l', 'o']

# Teach hihell -> ihello
x_data = [0,1,0,2,3,3] # hihell
y_data = [1,0,2,3,3,4] # ihello

# one hot encoding "hello":
one_hot = [[1,0,0,0,0],    # h
           [0,1,0,0,0],    # i
           [0,0,1,0,0],    # e
           [0,0,0,1,0],    # l
           [0,0,0,0,1]]    # o

x_one_hot = [one_hot[x] for x in x_data]

# As we have one batch of samples, we will change them to variables only once
inputs = autograd.Variable(torch.Tensor([x_one_hot]))
labels = autograd.Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5  # one-hot size
hidden_size = 5  # output from the LSTM. 5 to directly predict one-hot
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  # one-layer rnn


class RNN(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(RNN, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.batch_size = batch_size

        self.rnn = nn.RNN(input_size=5, hidden_size=5, batch_first=True)

    def forward(self, x):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size) for batch_first=True
        hidden_0 = autograd.Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))

        # Reshape input
        x.view(self.batch_size, self.sequence_length, self.input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # hidden_0: (num_layers * num_directions, batch, hidden_size)

        out, _ = self.rnn(x, hidden_0)
        return out.view(-1, num_classes)


# Instantiate RNN model
rnn = RNN(num_classes, input_size, hidden_size, num_layers)
print(rnn)

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
    outputs = rnn(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.data.item()))
    print("Predicted string:", ''.join(result_str))

print("Learning finished!")

RNN(
  (rnn): RNN(5, 5, batch_first=True)
)
epoch: 1, loss: 1.722
Predicted string: lohhhl
epoch: 2, loss: 1.546
Predicted string: lollll
epoch: 3, loss: 1.422
Predicted string: lollll
epoch: 4, loss: 1.271
Predicted string: ihllll
epoch: 5, loss: 1.120
Predicted string: ihelll
epoch: 6, loss: 0.979
Predicted string: ihelll
epoch: 7, loss: 0.852
Predicted string: ihello
epoch: 8, loss: 0.745
Predicted string: ihello
epoch: 9, loss: 0.679
Predicted string: ihello
epoch: 10, loss: 0.642
Predicted string: ihello
epoch: 11, loss: 0.616
Predicted string: ihello
epoch: 12, loss: 0.597
Predicted string: ihello
epoch: 13, loss: 0.582
Predicted string: ihello
epoch: 14, loss: 0.570
Predicted string: ihello
epoch: 15, loss: 0.560
Predicted string: ihello
epoch: 16, loss: 0.551
Predicted string: ihello
epoch: 17, loss: 0.542
Predicted string: ihello
epoch: 18, loss: 0.533
Predicted string: ihello
epoch: 19, loss: 0.525
Predicted string: ihello
epoch: 20, loss: 0.518
Predicted string: ihello
epoch

In [120]:
# input dimension 4 --> output dimension 2
cell = nn.RNN(input_size=4, hidden_size=2, batch_first=True)

# one letter input
inputs = autograd.Variable(torch.Tensor([[h,e,l,l,o],
                                         [e,l,l,o,h],
                                         [l,e,h,l,o]]))

# initializing the hidden state:
# (num_layers * num_directions, batch_size, hidden_size)
# in this case: only 1 layer and 1 direction, 1 batch and a hidden_size or output_size of 2
hidden = autograd.Variable(torch.randn(1,3,2))

# feeding one letter at a time
out, hidden = cell(inputs, hidden)
print(out.data)

tensor([[[ 0.0515, -0.8492],
         [ 0.2795, -0.9419],
         [ 0.4416, -0.8957],
         [ 0.4098, -0.8876],
         [-0.6697, -0.9033]],

        [[ 0.8689, -0.2063],
         [ 0.5361, -0.7273],
         [ 0.4454, -0.8597],
         [-0.6698, -0.8994],
         [-0.0468, -0.9692]],

        [[ 0.5448, -0.8665],
         [ 0.0948, -0.9403],
         [-0.3374, -0.9686],
         [ 0.5992, -0.9051],
         [-0.7109, -0.9037]]])


## Anderer Versuch